# BiSRe-LLM: Perturbation Experiment with openai-community/gpt2

In this notebook I set up the perturbation experiments for openai-community/gpt2. First, the general infrastructure for interacting with GPT2 is set up. Second, the prompt is designed as well as the iterative API requests to Huggingface to generate the answers to the according questions and answer options. In a third and last step, the answers of the model are stored in a dataframe for further analysis.

## Dataset

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

main_question = ["question"]
main_answer = ["list_answer_options"]

#bias_answer_columns = ["list_answer_no_refusal","list_answer_no_middle", "list_answer_middle", "list_answer_options_reversed", "list_answer_options", "original_personality_prime", "original_emotional_prime"]
#non_bias_question_columns = ["question_typo", "question_letter_swap","question_synonym","question_paraphrased", "question_keyboardtypo"]

# delayed experiments with middle category
bias_answer_columns = ["list_answer_middle"]
non_bias_question_columns = []

# read the preprocessed questionnaire
#questionnaire =  pd.read_excel('../assets/20241029_selected_questionnaire.xlsx', engine='openpyxl')
questionnaire =  pd.read_csv('../assets/20241029_selected_questionnaire.csv', sep=";")
# subset the questionnaire with the ten selected questions
questionnaire = questionnaire[questionnaire["selected"]==1]
print(questionnaire.shape)
questionnaire.head(2)

(62, 20)


,Unnamed: 0,question_ID,question,full_question_ID,list_answer_options,list_answer_no_refusal,len_answer_options,odd_answer_options,list_answer_no_middle,list_answer_middle,original_emotional_prime,original_personality_prime,list_answer_options_reversed,_merge,question_typo,question_letter_swap,question_synonym,question_paraphrased,question_keyboardtypo,selected
0,0,Q1,"Important in life: Family For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Family",Q1,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ic life: Fimily nor eaca jf the following aspecto, indicete how important it ir wn your liae. Would bou say it is very imporcant, rathes importano, not very imporgant ob not impodtant at all? Famizy","Impartont in life: Flmiay For each of the following sapects, indicate how important it is in your life. uoWld you yas it is evry important, ratreh important, ton very important or not important ta all? Family","Crucial in life: Family For each of the following aspects, indicate how significant it is in your life. Would you say it is very important, rather important, not very important or not at all important? Family","How important is family to you? Please rate its significance in your life on a scale of ""very important"" to ""not important at all"".","Imporyant in kife: Family For esch of rhe following aspects, indicate how important ut is un your lide. Would you say it ia very imporrant, rather importamt, nit very important ir nor important ay all? Fanily",1
1,1,Q2,"Important in life: Friends For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Friends",Q2,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather Important ', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ix life: Fpiends For xach of the following aspects, indicate how important iv iu in kour lifev Would dou say it os very importcnt, rather impoutant, nom vwry important hr not important xt bll? Friemds","Important ni :ifel Frisnde For each of the following aspects, indicaet 

## Experiments on Remote PC
The following section contains the experimental setup and scripts conducted on the remote PC.

In [2]:
#%pip install mistral-inference
%pip install transformers
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install 'accelerate>=0.26.0'
%pip install bitsandbytes
%pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.19.1%2Bcu118-cp39-cp39-linux_x86_64.whl (6.3 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.4.1%2Bcu118-cp39-cp39-linux_x86_64.whl (3.3 MB)
  Using cached https://download.pytorch.org/whl/cu118/torch-2.4.1%2Bcu118-cp39-cp39-linux_x86_64.whl (857.6 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_nvrtc_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (23.2 MB)
  Using cached https://download.pytorch.org/whl/cu118/nvidia_cuda_runtime_cu11-11.8.89-py3-none-manylinux1_x86_64.whl (875 kB)
  Using cac

In [11]:
# Just load once
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(
    "openai-community/gpt2", 
    quantization_config=quantization_config
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import torch

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="gpt2", device=0)
answer = pipe("Who are you?", max_new_tokens=10)

2024-10-30 17:53:49.776954: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 17:53:49.817265: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 17:53:49.817295: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 17:53:49.817320: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 17:53:49.825532: I tensorflow/core/platform/cpu_feature_g

In [6]:
# create empty response list which is transformed to a dataframe

response_list = []

# define rounds to be tested
rounds = 25

for round in range(0,rounds):
  print("Current round: ", round)
  torch.cuda.empty_cache()
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      for i,k in enumerate(bias_answer_columns):
          # first retrieve original question
          question = questionnaire.iloc[row]["question"]
          # second retrieve bias perturbed answer options from the questionnaire
          answer_options = questionnaire.iloc[row][k]
          messages = f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """
          answer = pipe(messages, max_length=1000, max_new_tokens=10)
          response = answer[0]["generated_text"]
          print(response)
          
          df_response_input = [round,question_ID, question, answer_options, k,  response]
          response_list.append(df_response_input)
          torch.cuda.empty_cache()
      for p,n in enumerate(non_bias_question_columns):
              # first retrieve non-bias perturbed question formats
              question = questionnaire.iloc[row][n]
              # second retrieve original answer option scale
              answer_options = questionnaire.iloc[row]["list_answer_options"]
              messages = f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """
              answer = pipe(messages, max_length=1000, max_new_tokens=10)
              response = answer[0]["generated_text"]
              print(response)
              df_response_input = [round,question_ID, question, answer_options, n, response]
              response_list.append(df_response_input)
              torch.cuda.empty_cache()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/te

Current round:  0
Answer the following question: Important in life: Family For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Family 
                        This is a list of possible answer options: ['1=Very important ', '2=Rather important', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']
                        You must pick one of the answer options. Only answer with the label.
                           * If the answer is'very important
Answer the following question: Important in life: Friends For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Friends 
                        This is a list of possible answer options: ['1=Very important ', '2=Rather Important ', '3=Neutral', '4=Not very important ', '5

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer the following question: Important in life: Leisure time For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Leisure time 
                        This is a list of possible answer options: ['1=Very important', '2=Rather important', '3=Neither important nor unimportant', '4=Not very important', '5=Not important at all', '-1=Don't know']
                        You must pick one of the answer options. Only answer with the label.
                         What does the item mean? Leisure time 
Answer the following question: Important in life: Politics For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Politics 
                        This is a list of possible answer options: ['1=Very important ', '2=Rather important ', '3=Neutral', '4=S

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https:/

...

Answer the following question: How often in countryselections: Election officials are fair In your view, how often do the following things occur in this countryselections? Election officials are fair 
                        This is a list of possible answer options: ['1=Very often ', '2=Fairly often ', '3=Sometimes ', '4=Not often ', '5=Not at all often', '-1=Don't know']
                        You must pick one of the answer options. Only answer with the label.
                            

[3=True
Answer the following question: How often in countryselections: Rich people buy elections In your view, how often do the following things occur in this countryselections? Rich people buy elections 
                        This is a list of possible answer options: ['1=Very often ', '2=Fairly often ', '3=Sometimes ', '4=Not often ', '5=Not at all often', '-1=Don't know']
                        You must pick one of the answer options. Only answer with the label.
                          It

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer the following question: Democracy: Religious authorities interpret the laws Please tell me for each of the following things how essential you think it is as a characteristic of democracy. Use this scale where a meansnot at all an essential characteristic of democracyand of means it definitely isan essential characteristic of democracy. Religious authorities interpret the laws 
                        This is a list of possible answer options: ['1=Not an essential feature ', '2 ', '3 ', '4 ', '5', '6=Somewhat essential', '7 ', '8 ', '9 ', '10', '11=Essential feature', '-1=Don't know']
                        You must pick one of the answer options. Only answer with the label.
                         | + 3 

"This is how
Answer the following question: Democracy: People choose their leaders in free elections Please tell me for each of the following things how essential you think it is as a characteristic of democracy. Use this scale where a meansnot at all an essential characteris

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer the following question: Democracy: Civil rights protect peoplea liberty against oppression Please tell me for each of the following things how essential you think it is as a characteristic of democracy. Use this scale where a meansnot at all an essential characteristic of democracyand of means it definitely isan essential characteristic of democracy. Civil rights protect peoplea liberty from state oppression 
                        This is a list of possible answer options: ['1=Not an essential feature ', '2 ', '3 ', '4 ', '5', '6=Somewhat essential', '7 ', '8 ', '9 ', '10', '11=Essential feature', '-1=Don't know']
                        You must pick one of the answer options. Only answer with the label.
                        
This answer will automatically be saved to your email
Answer the following question: Democracy: People obey their rulers Please tell me for each of the following things how essential you think it is as a characteristic of democracy. Use this scale whe

In [7]:
print("Length of responses: ", len(response_list))

Length of responses:  1550


In [8]:
# create a bias interaction experiment where there is a question-side and answer option-side bias
# here we use
perturbed_answer = ["list_answer_options_reversed"]
perturbed_question = "question_paraphrased"

for round in range(0,rounds):
  print("Current round: ", round)
  torch.cuda.empty_cache()
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      for i,k in enumerate(perturbed_answer):
          # first retrieve original question
          question = questionnaire.iloc[row]["question_paraphrased"]
          # second retrieve bias perturbed answer options from the questionnaire
          answer_options = questionnaire.iloc[row][k]
          messages = f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """
          answer = pipe(messages, max_length=1000, max_new_tokens=10)
          response = answer[0]["generated_text"]
          print(response)
          
          df_response_input = [round,question_ID, question, answer_options, "interaction_paraphrase_reversed",  response]
          response_list.append(df_response_input)
          torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Current round:  0
Answer the following question: How important is family to you?  Please rate its significance in your life on a scale of "very important" to "not important at all".   
                        This is a list of possible answer options: ["-1=Don't know", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']
                        You must pick one of the answer options. Only answer with the label.
                                  
Answer the following question: How important are friends in your life? Please indicate if they are very important, rather important, not very important, or not important at all.   
                        This is a list of possible answer options: ["-1=Don't know", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']
                        You must pick one of the answer options. Only answer with the label.
                         The "I" will be shown as a d

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer the following question: How important is leisure time in your life? Please rate its importance as very important, rather important, not very important, or not important at all.   
                        This is a list of possible answer options: ["-1=Don't know", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']
                        You must pick one of the answer options. Only answer with the label.
                           •   \ o -\ (
Answer the following question: How significant is politics in your life? Please rate its importance: very important, rather important, not very important, or not important at all.   
                        This is a list of possible answer options: ["-1=Don't know", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']
                        You must pick one of the answer options. Only answer with the label.
                         x You can have as m

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_

...

Answer the following question: In your opinion, how frequently do journalists provide impartial reporting on elections in this country?   
                        This is a list of possible answer options: ["-1=Don't know", '4=Not at all often', '3=Not often ', '2=Fairly often ', '1=Very often ']
                        You must pick one of the answer options. Only answer with the label.
                              This is an additional option
Answer the following question: In your opinion, how frequently are election officials impartial in this country's elections?   
                        This is a list of possible answer options: ["-1=Don't know", '4=Not at all often', '3=Not often ', '2=Fairly often ', '1=Very often ']
                        You must pick one of the answer options. Only answer with the label.
                                 .
Answer the following question: In your opinion, how frequently do wealthy individuals influence elections through financial contributio

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer the following question: A core component of democracy is the government's role in redistributing wealth by taxing the wealthy and providing financial assistance to the less fortunate. How essential do you believe this function is to a democratic society, on a scale of 1 to 5, where 1 represents "not at all essential" and 5 represents "definitely essential"?   
                        This is a list of possible answer options: ["-1=Don't know", '10=Essential feature', '9 ', '8 ', '7 ', '6 ', '5 ', '4 ', '3 ', '2 ', '1=Not an essential feature ']
                        You must pick one of the answer options. Only answer with the label.
                        
Bargaining is not necessarily necessary - this
Answer the following question: In a democracy, how important do you think it is for religious authorities to interpret the laws? Please rate the importance on a scale of 1 to 5, where 1 means "not at all essential" and 5 means "definitely essential."   
                       

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer the following question: How important is it for a democracy to provide state aid for unemployment?  On a scale of 1 to 5, where 1 means "not at all essential" and 5 means "definitely essential", please rate this characteristic.   
                        This is a list of possible answer options: ["-1=Don't know", '10=Essential feature', '9 ', '8 ', '7 ', '6 ', '5 ', '4 ', '3 ', '2 ', '1=Not an essential feature ']
                        You must pick one of the answer options. Only answer with the label.
                         

The above questions are not an exhaustive
Answer the following question: Democracy relies on civil rights to safeguard individual freedom from government tyranny.  How important do you consider each of the following aspects to be in a democracy, using this scale:  "a" means not essential at all, while "f" means absolutely essential?   
                        This is a list of possible answer options: ["-1=Don't know", '10=Essential feature', '9 ', '

In [9]:
print("Length of responses after interaction experiments: ", len(response_list))

Length of responses after interaction experiments:  3100


In [10]:
# create an empty dataframe which is filled with the model responses to the q&a combinations
response_columns = ["round","question_ID", "question", "answer_options", "type", "response"]
model_response = pd.DataFrame(response_list, columns=response_columns)
model_response["model_name"] = "gpt2"

In [11]:
from datetime import datetime

def get_date_string():
    # Get the current date
    now = datetime.now()
    
    # Format the date as YYYYMMDD
    date_string = now.strftime('%Y%m%d')
    
    return date_string

current_date = get_date_string()
model_response.to_csv(f"../assets/final_interviews/{current_date}_gpt2_selected_interviews.csv", encoding="utf-8", sep=";")